# Exercício 1 - Enunciado

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
    - A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. 
        - o modo `Start` inicia o funcionamento com os valores iniciais das velocidades
        - no modo `Free`  não existe qualquer força de travagem; 
        - no modo `Stopping` aplica-se a força de travagem alta; 
        - no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa) com pequeno atrito ao solo; 
        - no modo `Stopped` o veículo está imobilizado.
    - A componente contínua  do autómato usa variáveis contínuas $$\,V,v\,$$ para descrever a  `velocidade do corpo`   e a `velocidade linear das rodas`  ambas em relação so solo.
    - Assume-se que o sistema de travagem exerce uma força de atrito proporcional à diferença das duas velocidades.  A dinâmica contínua, as equações de fluxo, está descrita  abaixo.
    - Os “switchs” são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser  de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
    - É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

Faça 

1. Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos.
2. A condição de segurança estabelece que o sistema não permaneça no modo `free`  ou no modo `blocked` mais do que $$\,\tau\,$$ segundos.
3. Defina um SFOTS que modele a discretização do autómato híbrido.
4. Verifique nesse modelo
    1. Que as condições de segurança são invariantes do sistema
    2. Que o sistema atinge o estado `stopped` eventualmente.
    
Equações de Fluxo 

1. Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas. 
2. No contacto rodas/solo a força de compressão é dada pelo o peso $$P$$ que é constante e independente do modo. Tem-se $$f = a\, P$$ sendo  $$a$$ a constante de atrito; o valor de $$a$$ depende do modo: é baixa em `Blocked` e alta nos restantes.
3. No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à diferença de velocidades
$$F =  c\, (V-v)$$
      A  constante de proporcionalidade $$c$$ depende do modo: é elevada no modo `Stopping` e baixa nos outros.

4. As  equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`,
$$(\dot{V} \,=\, -F)\,\land\, (\dot{v} \,=\, -a\, P  + F)$$   
e , no modo `Blocked`,  a dinâmica do sistema é  regida por
                                                         $$(V = v) \;\land\; (\,\dot{v}\,=\, -a\, P\,)$$

6. Tanto no modo `Blocked`  como no modo `Free`  existe um “timer” que impede que o controlo aí permaneça mais do que $$\,\tau\,$$segundos.  Os     $$\mathsf{switch}(V,v,t,V',v',t')\,$$  nesses modos devem forçar esta condição. 
7. Todos os “switchs” devem ser construídos de  modo a impedir a existência de trajetórias de Zenão.
8. No instante inicial  o modo é  `Start` e  tem-se $$\,V = v\,=\,V_0$$  . A velocidade $$V_0$$ é  “input” do problema.

# Exercício 1 - Solução

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

START = Int(-1)
FREE = Int(0)
STOPPING = Int(1)
BLOCKED = Int(2)
STOPPED = Int(3)

MODE = {-1: "START", 0: "FREE", 1: "STOPPING", 2: "BLOCKED", 3: "STOPPED"}

In [2]:
def declare(i):
        s = {}
        s['t'] = Symbol('t'+str(i), REAL) # Time
        s['tau'] = Symbol('tau'+str(i), REAL) # Time in FREE and BLOCKED
        s['m'] = Symbol('m'+str(i), INT) # Mode
        s['V'] = Symbol('V'+str(i), REAL) # Velocity of body
        s['v'] = Symbol('v'+str(i), REAL) # Velocity of wheels
        s['F'] = Symbol('F'+str(i), REAL) # Braking force
        s['a'] = Symbol('a'+str(i), REAL) # Friction
        s['c'] = Symbol('c'+str(i), REAL) # Proportionality
        return s

In [3]:
def init(s, V0, aA, cB):
    return And(Equals(s['t'], Real(0)),
               Equals(s['tau'], Real(0)),
               Equals(s['m'], START),
               Equals(s['V'], Real(V0)),
               Equals(s['v'], Real(V0)),
               Equals(s['F'], Real(0)),
               Equals(s['a'], Real(aA)),
               Equals(s['c'], Real(cB))
              )

def trans(s, p, P, aA, aB, cA, cB, tau):
    # Untimed
    
    # starting
    starting = And(Equals(s['t'], p['t']),
                   Equals(s['m'], START),
                   Equals(s['m'], FREE)
                  )
    
    # breaking
    breaking = And(Equals(s['t'], p['t']),
                   Equals(s['m'], FREE),
                   Equals(p['m'], STOPPING),
                   Equals(s['F'], Times(s['c'], Minus(s['V'], s['v']))),
                   Equals(s['c'], Real(cB)),
                   GT(s['F'], Real(0))
                  )
    
    # blocking
    blocking = And(Equals(s['t'], p['t']),
                   Equals(s['m'], STOPPING),
                   Equals(p['m'], BLOCKED),
                   Equals(s['V'], s['v']),
                   Not(Equals(s['V'], Real(0)))
                  )
    
    # freeing
    freeing = And(Equals(s['t'], p['t']),
                  Equals(s['m'], BLOCKED),
                  Equals(s['m'], FREE),
                  Equals(s['F'], Times(s['c'], Minus(s['V'], s['v']))),
                  Equals(s['c'], Real(cB)),
                  Equals(s['F'], Real(0)),
                  Equals(p['tau'], Real(0))
                 )
    
    # ending
    ending = And(Equals(s['t'], p['t']),
                 Equals(s['m'], STOPPING),
                 Equals(s['m'], STOPPED),
                 Equals(s['V'], s['v']),
                 Equals(s['V'], Real(0))
                )
    
    untimed = Or(starting, breaking, blocking, freeing, ending)
    
    # Timed
    
    # common STOPPING and FREE
    commonSF = And(Equals(Plus(p['V'], -s['v'], 
                               Times(s['c'], s['V'], p['t']),
                               -Times(s['c'], s['V'], s['t']),
                               -Times(s['c'], s['v'], p['t']),
                               Times(s['c'], s['v'], s['t'])
                              ),
                          Real(0)
                         ),
                   Equals(Plus(p['V'],
                               -s['V'],
                               Times(s['a'], Real(P), p['t']),
                               -Times(s['c'], s['V'], p['t']),
                               Times(s['c'], s['v'], p['t']),
                               -Times(s['a'], Real(P), s['t']),
                               Times(s['c'], s['V'], s['t']),
                               -Times(s['c'], s['v'], s['t'])
                              ), 
                          Real(0)
                         )
                  )
    
    commonB = And(Equals(Plus(p['V'],
                              -s['V'],
                              Times(s['a'], Real(P), p['t']),
                              Times(s['a'], Real(P), p['t'])
                             ),
                         Real(0)
                        ),
                  Equals(s['V'],
                         s['v']
                        )
                 )
    
    # start    
    start = And(Equals(s['t'], Real(0)),
                Equals(s['V'], s['v']),
                Equals(s['V'], Real(V0))
               )
    
    # free
    free = And(LT(s['t'], p['t']),
                   LT(s['tau'], Real(tau)),
                   LT(s['tau'], p['tau']),
                   Equals(s['F'], Real(0)),
                   Equals(s['c'], Real(cB)),
                   Equals(s['F'], Times(s['c'], Minus(s['V'], s['v']))),
                   Equals(s['a'], Real(aA)),
                   commonSF
                  )
    
    # stopping
    stopping = And(Equals(s['F'], Times(s['c'], Minus(s['V'], s['v']))),
                   GT(s['F'], Real(0)),
                   LT(s['t'], p['t']),
                   Equals(s['tau'], Real(0)),
                   Equals(s['tau'], p['tau']),
                   Equals(s['c'], Real(cA)),
                   Equals(s['a'], Real(aA)),
                   commonSF
                  )
    
    # blocked
    blocked = And(Equals(s['F'], Times(s['c'], Minus(s['V'], s['v']))),
                  GT(s['F'], Real(0)),
                  LT(s['t'], p['t']),
                  LT(s['tau'], p['tau']),
                  LT(s['tau'], Real(tau)),
                  Equals(s['c'], Real(cB)),
                  Equals(s['a'], Real(aB)),
                  commonB
                 )
    # stopped
    stopped = And(Equals(s['V'], s['v']),
                  Equals(s['V'], Real(0))
                 )
    
    timed = Or(start, free, stopping, blocked, stopped)
    
    return Or(untimed, timed)

In [4]:
def print_vars(s, solver):
    for var in s:
        if s[var].get_type() == REAL:
            print(f"  {var} = {float(solver.get_py_value(s[var]))}")
        # if s[var].get_type() == INT:
        if var == "m":
            print(f"  {var} = {MODE[solver.get_py_value(s[var])]}")

def gera_traco(declare,init,trans,k, V0, P, aA, aB, cA, cB, tau):
    states = [declare(i) for i in range(k)]
    with Solver(name="z3") as solver:
        solver.add_assertion(init(states[0], V0, aA, cB))
        for i in range(k-1):
            solver.add_assertion(trans(states[i], states[i+1], P, aA, aB, cA, cB, tau))
        
        if solver.solve():
            for i,s in enumerate(states):
                print(f"> State {i}:")
                print_vars(s, solver)
        else:
            print("> Not feasible.")

In [5]:
V0 = 100
P = 1500
aA = 1
aB = 0.3
cA = 1
cB = 0.3
tau = 0.5
gera_traco(declare, init, trans, 10, V0, P, aA, aB, cA, cB, tau)

SolverReturnedUnknownResultError: 